In [ ]:
### With Partial dummification 

In [92]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

In [8]:
pd.options.display.max_columns=999

In [9]:
sample_submission_file = '../Data/sample_submission.csv'
train_file = '../Data/train.csv'
test_file = '../Data/test.csv'

In [10]:
pd.read_csv(sample_submission_file).head(2)

ID  PredictedProb
0   0            0.5
1   1            0.5

In [11]:
train_full = pd.read_csv(train_file)
test_full = pd.read_csv(test_file)

In [12]:
train=train_full[:]
test = test_full[:]

In [13]:
train.shape

(114321, 133)

In [14]:
test.shape

(114393, 132)

In [15]:
pd.DataFrame(pd.isnull(train).sum()).T

ID  target     v1     v2    v3     v4     v5     v6     v7     v8     v9  \
0   0       0  49832  49796  3457  49796  48624  49832  49832  48619  49851   

   v10    v11  v12    v13  v14    v15    v16    v17    v18    v19    v20  v21  \
0   84  49836   86  49832    4  49836  49895  49796  49832  49843  49840  611   

   v22    v23  v24    v25    v26    v27    v28    v29    v30   v31    v32  \
0  500  50675    0  48619  49832  49832  49832  49832  60110  3457  49832   

     v33  v34    v35    v36    v37  v38    v39  v40    v41    v42    v43  \
0  49832  111  49832  48624  49843    0  49836  111  49832  49832  49836   

     v44    v45    v46  v47    v48    v49  v50    v51  v52    v53    v54  \
0  49796  49832  48619    0  49796  49832   86  50678    3  49836  48619   

     v55   v56    v57    v58    v59    v60    v61  v62    v63    v64    v65  \
0  49832  6882  49832  49836  49796  49832  49796    0  48619  49796  49840   

   v66    v67    v68    v69    v70  v71  v72    v73  v74  v75    v76    v77  \
0    0  49832  49836  49895  48636    0    0  49836    0    0  49796  49832   

     v78  v79    v80    v81    v82    v83    v84    v85    v86    v87    v88  \
0  49895    0  49851  48624  48624  49832  49832  50682  49832  48663  49832   

     v89    v90  v91    v92    v93    v94    v95    v96    v97    v98    v99  \
0  48619  49836    3  49843  49832  49832  49843  49832  49843  48654  49832   

    v100   v101   v102   v103   v104   v105   v106  v107   v108   v109  v110  \
0  49836  49796  51316  49832  49832  48658  49796     3  48624  48624     0   

    v111  v112   v113  v114   v115   v116   v117   v118   v119   v120   v121  \
0  49832   382  55304    30  49895  49836  48624  49843  50680  49836  49840   

    v122   v123   v124  v125   v126   v127   v128  v129   v130   v131  
0  49851  50678  48619    77  49832  49832  48624     0  49843  49895

In [16]:
pd.DataFrame(pd.isnull(test).sum()).T

ID     v1     v2    v3     v4     v5     v6     v7     v8     v9  v10  \
0   0  49879  49839  3492  49839  48642  49879  49879  48636  49886   82   

     v11  v12    v13  v14    v15    v16    v17    v18    v19    v20  v21  v22  \
0  49881   84  49879    2  49881  49944  49839  49879  49887  49882  573  477   

     v23  v24    v25    v26    v27    v28    v29    v30   v31    v32    v33  \
0  50713    0  48636  49879  49879  49879  49879  60342  3492  49879  49879   

   v34    v35    v36    v37  v38    v39  v40    v41    v42    v43    v44  \
0  104  49879  48642  49887    0  49880  104  49879  49879  49881  49839   

     v45    v46  v47    v48    v49  v50    v51  v52    v53    v54    v55  \
0  49879  48636    0  49839  49879   84  50717    2  49881  48636  49879   

    v56    v57    v58    v59    v60    v61  v62    v63    v64    v65  v66  \
0  6918  49879  49880  49839  49879  49839    0  48636  49839  49882    0   

     v67    v68    v69    v70  v71  v72    v73  v74  v75    v76    v77    v78  \
0  49879  49880  49944  48647    0    0  49881    0    0  49839  49879  49944   

   v79    v80    v81    v82    v83    v84    v85    v86    v87    v88    v89  \
0    0  49886  48642  48642  49879  49879  50717  49879  48671  49879  48636   

     v90  v91    v92    v93    v94    v95    v96    v97    v98    v99   v100  \
0  49882    2  49887  49879  49879  49887  49879  49887  48673  49879  49880   

    v101   v102   v103   v104   v105   v106  v107   v108   v109  v110   v111  \
0  49839  51362  49879  49879  48665  49839     2  48642  48642     0  49879   

   v112   v113  v114   v115   v116   v117   v118   v119   v120   v121   v122  \
0   413  55358    24  49944  49881  48642  49887  50716  49880  49882  49886   

    v123   v124  v125   v126   v127   v128  v129   v130   v131  
0  50717  48636    84  49879  49879  48642     0  49887  49944

In [18]:
y_train=train.target

In [25]:
X_train = train.drop(labels=['target'], axis=1)
X_test = test

In [26]:
#### Removing v22
columns_categorical = list(X.dtypes[X_train.dtypes=='object'].index)
columns_categorical.remove('v22')
columns_categorical = pd.Index(columns_categorical)

In [27]:
columns_categorical

Index(['v3', 'v24', 'v30', 'v31', 'v47', 'v52', 'v56', 'v66', 'v71', 'v74',
       'v75', 'v79', 'v91', 'v107', 'v110', 'v112', 'v113', 'v125'],
      dtype='object')

In [28]:
X_train_categorical = X_train[columns_categorical]
X_test_categorical =X_test[columns_categorical]

In [29]:
X_train_categorical_dummies = pd.get_dummies(X_train_categorical)
X_test_categorical_dummies = pd.get_dummies(X_test_categorical)

In [30]:
X_train_categorical_dummies.shape

(114321, 364)

In [31]:
X_test_categorical_dummies.shape

(114393, 356)

In [37]:
columns_numerical = X.dtypes[X.dtypes!='object'].index

In [38]:
len(columns_numerical) + len(columns_categorical)

131

In [39]:
X_train_numerical = X_train[columns_numerical]
X_test_numerical = X_test[columns_numerical]

In [63]:
X_train_dummified = pd.concat([X_train_categorical_dummies,X_train_numerical], axis=1)
X_test_dummified = pd.concat([X_test_categorical_dummies,X_test_numerical], axis=1)

In [64]:
X_train_dummified.shape,X_test_dummified.shape

((114321, 477), (114393, 469))

In [65]:
common_columns = list(set(X_train_dummified.columns).intersection(X_test_dummified.columns))

In [66]:
X_train_dummified=X_train_dummified[common_columns]
X_test_dummified=X_test_dummified[common_columns]

In [67]:
X_train_dummified.shape

(114321, 457)

In [68]:
X_test_dummified.shape

(114393, 457)

In [75]:
X_train_dummified.sort_index(axis=1, inplace=True)

In [76]:
X_test_dummified.sort_index(axis=1, inplace=True)

In [77]:
X_train_dummified.drop(labels='ID', axis=1, inplace=True)

In [79]:
X_test_dummified.drop(labels='ID', axis=1, inplace=True)

In [78]:
X_train_dummified.head(1)

v1       v10       v100      v101      v102      v103      v104  \
0  1.335739  0.503281  19.470199  8.389237  2.757375  4.374296  1.574039   

       v105       v106  v107_A  v107_B  v107_C  v107_D  v107_E  v107_F  \
0  0.007294  12.579184       0       0       0       0       1       0   

   v107_G      v108      v109        v11  v110_A  v110_B  v110_C      v111  \
0       0  2.382692  3.930922  16.434108       0       1       0  0.433213   

   v112_A  v112_B  v112_C  v112_D  v112_E  v112_F  v112_G  v112_H  v112_I  \
0       0       0       0       0       0       0       0       0       0   

   v112_J  v112_K  v112_L  v112_M  v112_N  v112_O  v112_P  v112_Q  v112_R  \
0       0       0       0       0       0       1       0       0       0   

   v112_S  v112_T  v112_U  v112_V  v113_A  v113_AA  v113_AB  v113_AC  v113_AD  \
0       0       0       0       0       0        0        0        0        0   

   v113_AE  v113_AF  v113_AG  v113_AH  v113_AI  v113_AJ  v113_B  v113_C  \
0        0        0        0        0        0        0       0       0   

   v113_D  v113_E  v113_F  v113_G  v113_H  v113_I  v113_J  v113_L  v113_M  \
0       0       0       0       0       0       0       0       0       0   

   v113_N  v113_O  v113_P  v113_Q  v113_R  v113_S  v113_T  v113_U  v113_V  \
0       0       0       0       0       0       0       0       0       0   

   v113_W  v113_X  v113_Y  v113_Z       v114      v115     v116      v117  \
0       0       0       0       0  15.634907  2.857144  1.95122  6.592012   

       v118          v119       v12      v120      v121  v122     v123  \
0  5.909091 -6.297423e-07  6.085711  1.059603  0.803572     8  1.98978   

       v124  v125_A  v125_AA  v125_AB  v125_AC  v125_AD  v125_AE  v125_AF  \
0  0.035754       0        0        0        0        0        0        0   

   v125_AG  v125_AH  v125_AI  v125_AJ  v125_AK  v125_AL  v125_AM  v125_AN  \
0        0        0        0        0        0        0        0        0   

   v125_AO  v125_AP  v125_AQ  v125_AR  v125_AS  v125_AT  v125_AU  v125_AV  \
0        0        0        0        0        0        0        1        0   

   v125_AW  v125_AX  v125_AY  v125_AZ  v125_B  v125_BA  v125_BB  v125_BC  \
0        0        0        0        0       0        0        0        0   

   v125_BD  v125_BE  v125_BF  v125_BG  v125_BH  v125_BI  v125_BJ  v125_BK  \
0        0        0        0        0        0        0        0        0   

   v125_BL  v125_BM  v125_BN  v125_BO  v125_BP  v125_BQ  v125_BR  v125_BS  \
0        0        0        0        0        0        0        0        0   

   v125_BT  v125_BU  v125_BV  v125_BW  v125_BX  v125_BY  v125_BZ  v125_C  \
0        0        0        0        0        0        0        0       0   

   v125_CA  v125_CB  v125_CC  v125_CD  v125_CE  v125_CF  v125_CG  v125_CH  \
0        0        0        0        0        0        0        0        0   

   v125_CI  v125_CJ  v125_CK  v125_CL  v125_D  v125_E  v125_F  v125_G  v125_H  \
0        0        0        0        0       0       0       0       0       0   

   v125_I  v125_J  v125_K  v125_L  v125_M  v125_N  v125_O  v125_P  v125_Q  \
0       0       0       0       0       0       0       0       0       0   

   v125_R  v125_S  v125_T  v125_U  v125_V  v125_W  v125_X  v125_Y  v125_Z  \
0       0       0       0       0       0       0       0       0       0   

       v126      v127      v128  v129      v13      v130      v131        v14  \
0  1.804126  3.113719  2.024285     0  2.86683  0.636365  2.857144  11.636387   

        v15       v16      v17      v18       v19        v2        v20  \
0  1.355013  8.571429  3.67035  0.10672  0.148883  8.727474  18.869283   

        v21           v23  v24_A  v24_B  v24_C  v24_D  v24_E       v25  \
0  7.730923 -1.716131e-08      0      0      1      0      0  0.139412   

        v26       v27       v28       v29  v30_A  v30_B  v30_C  v30_D  v30_E  \
0  1.720818  3.393503  0.590122  8.880867      0      0      1

In [80]:
X_test_dummified.head(1)

v1       v10      v100      v101      v102      v103      v104  \
0  1.375465  1.312911  19.70803  4.186787  1.873945  4.129022  1.701894   

       v105       v106  v107_A  v107_B  v107_C  v107_D  v107_E  v107_F  \
0  0.004535  11.688117       0       1       0       0       0       0   

   v107_G     v108     v109        v11  v110_A  v110_B  v110_C      v111  \
0       0  1.50248  2.12171  16.851853       1       0       0  3.568773   

   v112_A  v112_B  v112_C  v112_D  v112_E  v112_F  v112_G  v112_H  v112_I  \
0       0       0       0       0       0       0       0       0       0   

   v112_J  v112_K  v112_L  v112_M  v112_N  v112_O  v112_P  v112_Q  v112_R  \
0       0       0       0       0       0       0       0       0       0   

   v112_S  v112_T  v112_U  v112_V  v113_A  v113_AA  v113_AB  v113_AC  v113_AD  \
0       0       0       1       0       0        0        0        0        0   

   v113_AE  v113_AF  v113_AG  v113_AH  v113_AI  v113_AJ  v113_B  v113_C  \
0        0        0        0        0        0        0       0       0   

   v113_D  v113_E  v113_F  v113_G  v113_H  v113_I  v113_J  v113_L  v113_M  \
0       0       0       0       0       0       0       0       0       0   

   v113_N  v113_O  v113_P  v113_Q  v113_R  v113_S  v113_T  v113_U  v113_V  \
0       0       0       0       0       0       0       0       0       0   

   v113_W  v113_X  v113_Y  v113_Z       v114       v115      v116      v117  \
0       0       0       0       0  12.246698  11.428572  1.276596  4.777902   

       v118          v119       v12      v120      v121      v122      v123  \
0  5.833334 -9.974950e-07  6.961173  1.459854  3.267328  7.619048  1.815241   

           v124  v125_A  v125_AA  v125_AB  v125_AC  v125_AD  v125_AE  v125_AF  \
0  1.112270e-07       0        0        0        0        0        0        1   

   v125_AG  v125_AH  v125_AI  v125_AJ  v125_AK  v125_AL  v125_AM  v125_AN  \
0        0        0        0        0        0        0        0        0   

   v125_AO  v125_AP  v125_AQ  v125_AR  v125_AS  v125_AT  v125_AU  v125_AV  \
0        0        0        0        0        0        0        0        0   

   v125_AW  v125_AX  v125_AY  v125_AZ  v125_B  v125_BA  v125_BB  v125_BC  \
0        0        0        0        0       0        0        0        0   

   v125_BD  v125_BE  v125_BF  v125_BG  v125_BH  v125_BI  v125_BJ  v125_BK  \
0        0        0        0        0        0        0        0        0   

   v125_BL  v125_BM  v125_BN  v125_BO  v125_BP  v125_BQ  v125_BR  v125_BS  \
0        0        0        0        0        0        0        0        0   

   v125_BT  v125_BU  v125_BV  v125_BW  v125_BX  v125_BY  v125_BZ  v125_C  \
0        0        0        0        0        0        0        0       0   

   v125_CA  v125_CB  v125_CC  v125_CD  v125_CE  v125_CF  v125_CG  v125_CH  \
0        0        0        0        0        0        0        0        0   

   v125_CI  v125_CJ  v125_CK  v125_CL  v125_D  v125_E  v125_F  v125_G  v125_H  \
0        0        0        0        0       0       0       0       0       0   

   v125_I  v125_J  v125_K  v125_L  v125_M  v125_N  v125_O  v125_P  v125_Q  \
0       0       0       0       0       0       0       0       0       0   

   v125_R  v125_S  v125_T  v125_U  v125_V  v125_W  v125_X  v125_Y  v125_Z  \
0       0       0       0       0       0       0       0       0       0   

       v126      v127      v128  v129       v13      v130      v131  \
0  1.292368  3.903345  1.485925     0  3.107462  2.333334  1.428572   

         v14      v15       v16       v17       v18       v19         v2  \
0  12.230137  1.77305  2.857142  3.493816  0.096826  0.225481  11.361141   

         v20       v21           v23  v24_A  v24_B  v24_C  v24_D  v24_E  \
0  17.910227  7.437806  9.558310e-07      0      0      0      0      1   

       v25       v26       v27       v28       v29  v30_A  v30_B  v30_C  \
0  0.06918  0.954152  4.386616  0.573017  6.988848      0      0 

In [82]:
X_train = X_train_dummified.values
X_test = X_test_dummified.values

In [83]:
X_train.shape, X_test.shape

((114321, 456), (114393, 456))

In [81]:
### Filling missing values

In [84]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X_train)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [85]:
X_train = imp.transform(X_train)

In [89]:
X_test = imp.transform(X_test)

In [90]:
X_train.shape

(114321, 456)

In [91]:
X_test.shape

(114393, 456)

In [93]:
logreg = LogisticRegression()

In [97]:
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [98]:
score = cross_val_score(logreg, X_train, y_train)

In [100]:
score.mean()

0.77424970040499996

In [106]:
y_test = logreg.predict_proba(X_test)[:,1]

In [107]:
y_test

array([ 0.4504562 ,  0.9111789 ,  0.7125162 , ...,  0.82309818,
        0.85987007,  0.47939976])

In [109]:
y_submission = pd.DataFrame(np.zeros((y_test.shape[0],2)), columns = pd.read_csv(sample_submission_file).columns)

In [ ]:
### Submission 1 : 0.48211

In [112]:
y_submission['ID']=test['ID']
y_submission['PredictedProb']=y_test

In [115]:
y_submission.to_csv('../Submissions/submission1.csv', sep=',', index=False)

In [116]:
###Augmentation de la probabilité MAUVAIS

In [122]:
y_submission.loc[y_submission.PredictedProb>0.9,'PredictedProb']=1

In [133]:
y_submission.to_csv('../Submissions/submission2.csv', sep=',', index=False)

In [134]:
y_submission.shape

(114393, 2)

In [ ]:
### Class Weight = Auto (-> 0.60)

In [135]:
logreg = LogisticRegression(class_weight='balanced')

In [136]:
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [137]:
y_test = logreg.predict_proba(X_test)[:,1]

In [138]:
y_submission['PredictedProb']=y_test

In [139]:
y_submission.to_csv('../Submissions/submission3.csv', sep=',', index=False)

In [140]:
y_submission

ID  PredictedProb
0            0       0.194849
1            1       0.743868
2            2       0.439262
3            7       0.290468
4           10       0.421015
5           11       0.310623
6           13       0.832445
7           14       0.313330
8           15       0.615824
9           16       0.632994
10          17       0.407197
11          18       0.753868
12          19       0.709335
13          20       0.694427
14          25       0.584274
15          26       0.689147
16          29       0.598266
17          38       0.932440
18          41       0.823069
19          44       0.325157
20          45       0.165060
21          47       0.408608
22          48       0.489653
23          49       0.643274
24          50       0.637136
25          53       0.298671
26          56       0.232004
27          59       0.608073
28          60       0.434485
29          64       0.628513
...        ...            ...
114363  228650       0.305799
114364  228656       0.613323
114365  228661       0.278823
114366  228662       0.722977
114367  228664       0.674211
114368  228665       0.827253
114369  228667       0.589251
114370  228669       0.754111
114371  228672       0.500811
114372  228674       0.394172
114373  228675       0.226984
114374  228676       0.824629
114375  228678       0.375915
114376  228679       0.442735
114377  228681       0.884591
114378  228685       0.480329
114379  228686       0.781179
114380  228687       0.466433
114381  228689       0.587148
114382  228690       0.547429
114383  228692       0.490836
114384  228693       0.414409
114385  228694       0.515956
114386  228696       0.581056
114387  228698       0.676563
114388  228700       0.650851
114389  228703       0.182112
114390  228704       0.585873
114391  228706       0.672809
114392  228709       0.214149

[114393 rows x 2 columns]